In [ ]:
ver = 'v10'

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import *
from sklearn.preprocessing import *
from sklearn.metrics import *
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

import gc
from tqdm import *

In [ ]:
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')
ss = pd.read_csv('sample_submission.csv')

In [ ]:
df = pd.concat([train, test]).reset_index(drop = True)

In [ ]:
df.nunique()

Area_Code        30900
Locality_Code       22
Region_Code       2185
Height             110
Diameter           151
Class                8
Species            263
dtype: int64

In [ ]:
df['Region_Species'] = pd.factorize(df['Region_Code'].astype('str') + df['Species'].astype('str'))[0]
df['Region_Species'].nunique()

17667

In [ ]:
df['height*diameter'] = df['Height'] * df['Diameter']
df['height/diameter'] = df['Height'] / df['Diameter']
df['log_diameter'] = np.log(df['Diameter'])

/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:679: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [ ]:
fts = ['Area_Code','Locality_Code','Region_Code','Height','Diameter', 'Species', 'Region_Species']

In [ ]:
for f in fts:
  df[f + '_freq'] = df[f].map(df[f].value_counts(normalize = True))

In [ ]:
cat_fts = ['Area_Code', 'Locality_Code', 'Region_Code', 'Species', 'Region_Species', 'Height', 'Diameter']
num_fts_2 = ['Height', 'Diameter','height*diameter', 'height/diameter', 'Locality_Code_freq', 'Species_freq']

for g in tqdm_notebook(cat_fts):

  num_fts = [c for c in num_fts_2 if c != g]
  grp = df.groupby(g)[num_fts].mean()
  grp.columns = [c + f'_grpd_by_{g}_mean' for c in grp.columns]
  df = pd.merge(df, grp, on = g, how = 'left')

  grp = df.groupby(g)[num_fts].std()
  grp.columns = [c + f'_grpd_by_{g}_std' for c in grp.columns]
  df = pd.merge(df, grp, on = g, how = 'left')

  grp = df.groupby(g)[num_fts].min()
  grp.columns = [c + f'_grpd_by_{g}_min' for c in grp.columns]
  df = pd.merge(df, grp, on = g, how = 'left')

  grp = df.groupby(g)[num_fts].max()
  grp.columns = [c + f'_grpd_by_{g}_max' for c in grp.columns]
  df = pd.merge(df, grp, on = g, how = 'left')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


In [ ]:
fts = ['Area_Code','Locality_Code','Region_Code','Height','Diameter', 'Species', 'Region_Species']
for g in tqdm_notebook(fts):
  rf = [c for c in fts if c != g]
  grp = df.groupby(g)[rf].nunique()
  grp.columns = [c + f'_grpd_by_{g}_nunique' for c in grp.columns]
  df = pd.merge(df, grp, on = g, how = 'left')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [ ]:
train, test = df[:train.shape[0]].reset_index(drop = True), df[train.shape[0]:].reset_index(drop = True)

In [ ]:
features = [c for c in train.columns if c not in ['Class', "Area_Code", "Region_Code", 'Region_Species']]

In [ ]:
len(features)

216

In [ ]:
oofs = np.zeros((len(train), 8))
preds = np.zeros((len(test), 8))
N_SPLITS = 15
folds = StratifiedKFold(N_SPLITS, shuffle = True, random_state = 77)

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train, train['Class'])):

  print(f'\n\n Fold {fold_} \\n\n')
  X_trn, y_trn = train[features].iloc[trn_idx], train["Class"][trn_idx]
  X_val, y_val = train[features].iloc[val_idx], train["Class"][val_idx]

  # clf = LGBMClassifier(n_estimators = 10000, learning_rate = 0.05, colsample_bytree = 0.5, reg_alpha = 0.5, reg_lambda = 1, random_state = 2)
  clf = CatBoostClassifier(iterations = 5000, learning_rate = 0.06, random_state=2, reg_lambda=0.5, eval_metric='MultiClass', task_type = 'GPU')

  clf.fit(X_trn, y_trn, eval_set = [(X_val, y_val)], early_stopping_rounds = 150, verbose = 50)

  oofs[val_idx] = clf.predict_proba(X_val)
  preds += clf.predict_proba(test[features]) / N_SPLITS




 Fold 0 \n

0:	learn: 1.9185089	test: 1.9243102	best: 1.9243102 (0)	total: 12.5ms	remaining: 1m 2s
50:	learn: 0.8404664	test: 0.9106290	best: 0.9106290 (50)	total: 548ms	remaining: 53.2s
100:	learn: 0.7348459	test: 0.8448110	best: 0.8448110 (100)	total: 1.05s	remaining: 50.9s
150:	learn: 0.6753437	test: 0.8176122	best: 0.8176122 (150)	total: 1.58s	remaining: 50.7s
200:	learn: 0.6287233	test: 0.7989932	best: 0.7989932 (200)	total: 2.08s	remaining: 49.7s
250:	learn: 0.5900514	test: 0.7874514	best: 0.7874514 (250)	total: 2.61s	remaining: 49.4s
300:	learn: 0.5553369	test: 0.7750005	best: 0.7750005 (300)	total: 3.12s	remaining: 48.8s
350:	learn: 0.5255657	test: 0.7704437	best: 0.7704437 (350)	total: 3.65s	remaining: 48.3s
400:	learn: 0.4984768	test: 0.7635515	best: 0.7635515 (400)	total: 4.17s	remaining: 47.9s
450:	learn: 0.4733966	test: 0.7579688	best: 0.7579688 (450)	total: 4.69s	remaining: 47.3s
500:	learn: 0.4509757	test: 0.7554362	best: 0.7551304 (491)	total: 5.2s	remaining: 46.7s
55

In [ ]:
oof_val_score = log_loss(train['Class'], oofs)
print(f'Final Log loss: {oof_val_score}')

Final Log loss: 0.7257320709221037


In [ ]:
ss[ss.columns.tolist()] = preds

In [ ]:
SUB_FILE_NAME = f'mft_{ver}.csv'
ss.to_csv(SUB_FILE_NAME, index=False)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>